In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use('ggplot')
from sklearn.cross_validation import train_test_split
import time
from sklearn.externals import joblib

%matplotlib inline

order = pd.read_excel('C:/Users/ezarpkm/Desktop/CaseData/OrderRows.xlsx')
productinfo = pd.read_excel('C:/Users/ezarpkm/Desktop/CaseData/ProductInfo.xlsx')
productiddf = pd.read_excel('C:/Users/ezarpkm/Desktop/CaseData/Sheet2.xlsx')
#order1 = pd.read_excel(order, 'Sheet1')
#order2 = pd.read_excel(order, 'Sheet2')

C:\Users\ezarpkm\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
productinfo.head()

,productno,productmanufacturerid,category
0,1,2,Hushållsapparater - Smarta hem - Övervakningsk...
1,2,336,Hushållsapparater - Hushållsapparater - Köksradio
2,3,383,Hushållsapparater - Hushållsapparater - Hörlurar
3,4,383,Hushållsapparater - Hushållsapparater - Hörlurar
4,5,383,Hushållsapparater - Hushållsapparater - Hörlurar


In [3]:
filterprod = pd.merge(productinfo, productiddf, on=['productno'])
filterorder = pd.merge(filterprod,order, on = ['productno'])
filterorder = filterorder.drop('createdon',axis =1)
filterorder.productid = filterorder.productid.astype(str) 
filterorder.dtypes

filterorder['category'] = filterorder['category'].map(str) + " -> " + filterorder['productid']
filterorder = filterorder.drop(['productid','orderno','productno'], axis = 1)
filterorder

,productmanufacturerid,category,quantity
0,2,Hushållsapparater - Smarta hem - Övervakningsk...,1
1,2,Hushållsapparater - Smarta hem - Övervakningsk...,1
2,2,Hushållsapparater - Smarta hem - Övervakningsk...,1
3,2,Hushållsapparater - Smarta hem - Övervakningsk...,2
4,2,Hushållsapparater - Smarta hem - Övervakningsk...,1
5,2,Hushållsapparater - Smarta hem - Övervakningsk...,1
6,2,Hushållsapparater - Smarta hem - Övervakningsk...,4
7,383,Hushållsapparater - Hushållsapparater - Hörlur...,1
8,383,Hushållsapparater - Hushållsapparater - Köksra...,1
9,383,Hushållsapparater - Hushållsapparater - Köksra...,1


In [4]:
#aggregation
ford = filterorder.groupby(['category']).agg({'quantity': 'count'}).reset_index()
grouped_sum = ford['quantity'].sum()
ford['percentage']  = ford['quantity'].div(grouped_sum)*100

ford = ford.sort_values(['quantity', 'category'], ascending = [0,1])
data = ford
data

,category,quantity,percentage
5344,Hushållsapparater - Hushållsapparater - Sous-V...,3439,0.495812
35982,Tillredning - Redskap - Grönsakssvarvar -> 284...,3335,0.480818
41830,Utomhusmatlagning - Grillar - Grillredskap -> ...,2766,0.398784
37962,Tillredning - Redskap - Sifoner -> 27297790,2459,0.354523
34326,Tillredning - Kok - Grytset -> 28209300,2427,0.349909
42263,Utomhusmatlagning - Grillar - Stekhällar -> 26...,2205,0.317902
41087,Tillredning - Stekkärl - Stekpannor -> 27518298,2186,0.315163
26056,Servering - Glas/Muggar - Vinglas -> 27535255,2042,0.294402
36968,Tillredning - Redskap - Mandoliner -> 26685240,1903,0.274362
5922,Hushållsapparater - Hushållsapparater - Våffel...,1876,0.270469


In [5]:
#unique product productmanufacturerid
prod_manu_id = filterorder['category'].unique()
prod_manu_id

array(['Hushållsapparater - Smarta hem - Övervakningskameror -> 33799768',
       'Hushållsapparater - Hushållsapparater - Hörlurar -> 33031553',
       'Hushållsapparater - Hushållsapparater - Köksradio -> 33031555',
       ..., 'Inredning - Textil - Grytlappar -> 43083888',
       'Inredning - Textil - Dukar & Löpare -> 47992872',
       'Servering - Barredskap - Glasunderlägg -> 51401043'], dtype=object)

In [6]:
#create a product recommender
train_data, test_data = train_test_split(filterorder, test_size = 0.20, random_state = 0)
print(train_data.head(5))

        productmanufacturerid  \
653551                  12304   
587888                   9948   
376102                   7831   
229541                   7717   
533142                   8900   

                                                 category  quantity  
653551       Servering - Glas/Muggar - Muggar -> 29011757         1  
587888  Hushållsapparater - Hushållsapparater - Vatten...         1  
376102     Tillredning - Stekkärl - Plättlagg -> 27964565         1  
229541  Hushållsapparater - Hushållsapparater - Slow C...         1  
533142        Inredning - Textil - Grytlappar -> 28078498         1  


In [7]:
import rec as Recommenders

In [8]:
pm = Recommenders.popularity_recommender_py()
#pm.create(train_data, 'category', 'productid')

pm.create(train_data,'productmanufacturerid','category')

In [9]:
#The recommendation is a popularity based model as of now
#Intended to be used as a black box model
#Here, we didn't have any user information with us and we were given the task to recommend products once it has been ordered
#or added to the cart.


product_id_for_rec = '8068'


In [10]:
pm.recommend(product_id_for_rec)

,productmanufacturerid,category,score,Rank
5049,8068,Hushållsapparater - Hushållsapparater - Sous-V...,2732,1.0
33612,8068,Tillredning - Redskap - Grönsakssvarvar -> 284...,2682,2.0
39254,8068,Utomhusmatlagning - Grillar - Grillredskap -> ...,2213,3.0
35523,8068,Tillredning - Redskap - Sifoner -> 27297790,1980,4.0
32023,8068,Tillredning - Kok - Grytset -> 28209300,1953,5.0
38543,8068,Tillredning - Stekkärl - Stekpannor -> 27518298,1760,6.0
39666,8068,Utomhusmatlagning - Grillar - Stekhällar -> 26...,1753,7.0
24184,8068,Servering - Glas/Muggar - Vinglas -> 27535255,1650,8.0
34567,8068,Tillredning - Redskap - Mandoliner -> 26685240,1531,9.0
5608,8068,Hushållsapparater - Hushållsapparater - Våffel...,1499,10.0


# Recommendation with some added bit of personalization

In [11]:
train_data_item, test_data_item = train_test_split(productinfo, test_size = 0.20, random_state = 0)
print(train_data_item.head(5))

       productno  productmanufacturerid  \
15405      15406                   5042   
78970      78971                   8173   
80114      80115                   8186   
69645      69646                   8047   
44158      44159                   7103   

                                            category  
15405           Servering - Glas/Muggar - Dricksglas  
78970          Badrum - Badrumstextilier - Handdukar  
80114              Servering - Tallrikar - Tallrikar  
69645  Möbler - Möbler - Hyllor & Förvaring - Hyllor  
44158    Inredning - Belysning - Taklampor & Pendlar  


In [12]:
prid = productinfo['productmanufacturerid'].unique()
catitems = productinfo['category'].unique()
len(catitems)

466

In [15]:
item_sim = Recommenders.item_similarity_recommendation()

item_sim.create(train_data_item,'productmanufacturerid','category')
productmanufacturerid

1768

In [19]:
productmanufacturerid = prid[44]

In [20]:
cat_items = item_sim.get_manufacturer_items(productmanufacturerid)
print("------------------------------------------------------------------------------------")
print("Training data for the manufacturere Id: %s" % productmanufacturerid)
print("------------------------------------------------------------------------------------")


for cat_item in cat_items:
    print (cat_item)
print("----------------------------------------------------------------------")
print("Recommendation process going on:")
print("----------------------------------------------------------------------")

item_sim.recommend(productmanufacturerid)

------------------------------------------------------------------------------------
Training data for the manufacturere Id: 1773
------------------------------------------------------------------------------------
Hushållsapparater - Hushållsapparater - Luftrenare
Hushållsapparater - Hushållsapparater - Personvågar
Hushållsapparater - Hushållsapparater - Massagedynor
Import
Inredning - Textil - Plädar & Filtar
Hushållsapparater - Hushållsapparater - Rakapparater och trimmers
----------------------------------------------------------------------
Recommendation process going on:
----------------------------------------------------------------------
Number of unique categories for the manufacturer : 6
no. of unique products in the training set: 464
Non zero values in cooccurence_matrix :1396


,Manufacturer Id,Products,Score,Rank
0,1773,Hushållsapparater - Hushållsapparater - Hårfönar,0.220338,1
1,1773,Hushållsapparater - Hushållsapparater - Lock- ...,0.203289,2
2,1773,Hushållsapparater - Hushållsapparater - Strykjärn,0.112678,3
3,1773,Hushållsapparater - Hushållsapparater - Råsaft...,0.106601,4
4,1773,Hushållsapparater - Hushållsapparater - Elvispar,0.105755,5
5,1773,Hushållsapparater - Hushållsapparater - Övriga...,0.104176,6
6,1773,Hushållsapparater - Hushållsapparater - Fritöser,0.102946,7
7,1773,Hushållsapparater - Hushållsapparater - Juicem...,0.100668,8
8,1773,Hushållsapparater - Hushållsapparater - Dammsu...,0.094812,9
9,1773,Inredning - Textil - Kuddar,0.094617,10


In [23]:
item_sim.get_similar_items('Inredning - Textil - Kuddar')

no. of unique categories in the training set: 464
Non zero values in cooccurence_matrix :0


,Manufacturer Id,Products,Score,Rank
0,,Hushåll - Disk & köksrengöring,0.0,1
1,,Barn - Utomhusleksaker - Gungställningar,0.0,2
2,,Tillredning,0.0,3
3,,Inredning - Tillbehör - Tofflor & Väskor,0.0,4
4,,Badrum - Badrumsmöbler - Handdukstorkar,0.0,5
5,,Hushållsapparater - Hushållsapparater - Kokpla...,0.0,6
6,,Hushållsapparater - Tillbehör,0.0,7
7,,Barn - Servering - Glas & Muggar,0.0,8
8,,Servering,0.0,9
9,,Möbler - Möbler - Barnmöbler,0.0,10
